# Базовые библиотеки

In [ ]:
import pandas as pd
import os
import warnings
warnings.simplefilter('ignore')

In [ ]:
import data_extracting
import merging
import visualization

In [ ]:
DATA_DIR = 'Airlines'
DATA_EXPORTED = 'data'

# Извлечение данных в формат CSV
Запускать только если ещё нет экспортированных данных, так как некоторые функции могут работать существенное время

In [ ]:
os.mkdir(DATA_EXPORTED)

In [ ]:
data = data_extracting.extractBoardingData(f'{DATA_DIR}/BoardingData.csv')
data.to_csv(f'{DATA_EXPORTED}/BoardingData.csv', index=False)
del data

In [ ]:
data = data_extracting.extractAirlinesData(f'{DATA_DIR}/PointzAggregator-AirlinesData.xml')
data.to_csv(f'{DATA_EXPORTED}/PointzAggregator-AirlinesData.csv', index=False)
del data

In [ ]:
data = data_extracting.extractBoardingPass(f'{DATA_DIR}/YourBoardingPassDotAero.zip', clear_temp = True)
data.to_csv(f'{DATA_EXPORTED}/YourBoardingPassDotAero.csv', index=False)
del data

In [ ]:
data = data_extracting.extractFrequentFlyerForumProfiles(f'{DATA_DIR}/FrequentFlyerForum-Profiles.json')
data[0].to_csv(f'{DATA_EXPORTED}/FrequentFlyerForum-Profiles_Base.csv', index=False)
data[1].to_csv(f'{DATA_EXPORTED}/FrequentFlyerForum-Profiles_Flight.csv', index=False)
data[2].to_csv(f'{DATA_EXPORTED}/FrequentFlyerForum-Profiles_LoyalityProgram.csv', index=False)
del data

In [ ]:
data = data_extracting.extractSirenaExportFixed(f'{DATA_DIR}/Sirena-export-fixed.tab')
data.to_csv(f'{DATA_EXPORTED}/Sirena-export-fixed.csv', index=False)
del data

In [ ]:
data = data_extracting.extractSkyTeamExchange(f'{DATA_DIR}/SkyTeam-Exchange.yaml')
data.to_csv(f'{DATA_EXPORTED}/SkyTeam-Exchange.csv', index=False)
del data

In [ ]:
data = data_extracting.extractSkyteamTimetable(f'{DATA_DIR}/Skyteam_Timetable.pdf')
data.to_csv(f'{DATA_EXPORTED}/Skyteam_Timetable.csv', index=False)
del data

# Обработка данных

## 1. Обработка данных, с паспортами
Слияние, очистка и обработка данных из [этого](data/Sirena-export-fixed.csv) и [этого](data/BoardingData.csv) файлов

In [ ]:
df_sirena = pd.read_csv('data/Sirena-export-fixed.csv')

In [ ]:
df_boarding = pd.read_csv('data/BoardingData.csv', sep = ';')
df_boarding['PassangerName'] = df_boarding['PassengerLastName'] + ' ' + df_boarding['PassengerFirstName']

In [ ]:
# Проверка - нет ли людей, менявших имя, или указавших разные данные
dup1 = df_boarding[['PassengerDocument','PassangerName']].groupby('PassengerDocument')['PassangerName'].nunique()
dup1 = dup1[dup1 > 1].index.tolist()
print(f'Странные паспорта: {dup1}')
if len(dup1) > 0:
    print(df_boarding[df_boarding['PassengerDocument'].isin(dup1)][['PassangerName', 'PassengerDocument', 'PassengerBirthDate']].drop_duplicates())

In [ ]:
data1, docs = merging.mergeDataPasports(df_sirena, df_boarding)

In [ ]:
data1['ID'] = "pass_" + data1['PassengerDocument']
data1.rename(columns={'Dest':'To','FlightDate':'Date'}, inplace=True)

In [ ]:
print('Соответствие загранпаспортов и паспортов РФ:')
docs

## 2. Обработка данных, на основе уникальных номеров программ лояльности

In [ ]:
df_airlines = pd.read_csv('data/PointzAggregator-AirlinesData.csv')
df_airlines['uid'] = df_airlines['uid'].apply(str)

In [ ]:
df_exchange = pd.read_csv('data/SkyTeam-Exchange.csv')

In [ ]:
df_forum_b = pd.read_csv('data/FrequentFlyerForum-Profiles_Base.csv')
df_forum_f = pd.read_csv('data/FrequentFlyerForum-Profiles_Flight.csv')
df_forum_lp = pd.read_csv('data/FrequentFlyerForum-Profiles_LoyalityProgram.csv')
df_forum = (df_forum_b,df_forum_f,df_forum_lp)

In [ ]:
data2, strange, ids = merging.mergeLoyality(df_exchange, df_forum, df_airlines)

In [ ]:
print('Нехорошие люди, использовавшие чужие программы лояльности:')
strange

## 3. Слияние всего вместе

In [ ]:
# Соответствие ников, uid и паспорта
ids = merging.mergeLoyalityIdNickPasports(df_sirena, ids)

In [ ]:
# Данные, о полётах из второй части данных
data = pd.merge(data2, ids[['ID','FFKey']], on='FFKey', how='left')[['ID', 'Date','From','To']]
data = data.dropna()

In [ ]:
# Объединение всех данных о полётах вместе, для дальнейшей отрисовки
data = pd.concat([data, data1[['ID','Date','From','To']]])

In [ ]:
data

# Отрисовка графа полётов

In [ ]:
airports = pd.read_csv('data/Airports.csv')

In [ ]:
data = visualization.get_dataframe_for_work(data, airports)

In [ ]:
visualization.create_flight_graph_app(data)